<a href="https://colab.research.google.com/github/yashmaurya01/nanoGPT-experiments/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-12-28 05:55:32--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.3’

input.txt.3         100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2024-12-28 05:55:32 (28.4 MB/s) - ‘input.txt.3’ saved [1115394/1115394]



In [38]:
!pip install torch numpy tqdm

In [39]:
with open('input.txt', 'r') as file:
    text = file.read()

print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [40]:
print(len(text))

1115394


In [41]:
chars = sorted(list(set(text)))
print(''.join(chars))
vocab_size = len(chars)
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [42]:
##character level language model
import torch

stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

def encode(s):
    return [stoi[c] for c in s]

def decode(l):
    if isinstance(l, torch.Tensor):
        l = l.tolist()
    if isinstance(l, int):
        return itos[l]
    return ''.join([itos[i] for i in l])

print(encode('hii my name is yash'))
print(decode(encode('hii my name is yash')))



[46, 47, 47, 1, 51, 63, 1, 52, 39, 51, 43, 1, 47, 57, 1, 63, 39, 57, 46]
hii my name is yash


In [43]:
# import tiktoken

# enc = tiktoken.get_encoding('gpt2')

# print(enc.encode('yash'))
# print(enc.decode(enc.encode('yash')))

In [44]:
#encode the shakespeare text

import torch
from tqdm import tqdm

data = torch.tensor(encode(text), dtype=torch.long) #torch.long is equivalent to torch.int64
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [45]:
#train-val split 90-10
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

print(train_data.shape, val_data.shape)

torch.Size([1003854]) torch.Size([111540])


In [46]:
block_size = 8
decode(train_data[block_size:block_size+1])

't'

In [47]:
x = train_data[:block_size+1]
y = train_data[1:block_size+2]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {decode(context)} the target is {decode(target)}")

when input is F the target is i
when input is Fi the target is r
when input is Fir the target is s
when input is Firs the target is t
when input is First the target is  
when input is First  the target is C
when input is First C the target is i
when input is First Ci the target is t


In [48]:
# Device configuration
import torch
device = torch.device("cpu")  # Start with CPU as default
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
print(f"Using device: {device}")

Using device: cuda


In [49]:
torch.manual_seed(1337)
batch_size = 64
block_size = 256
n_embed = 384
num_heads = 6
n_layer = 6
dropout = 0.2
max_iters = 2000
eval_iters = 200
lr = 3e-4

def get_batch(split):
    data = train_data if split == 'train' else val_data
    random_indices = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in random_indices])
    y = torch.stack([data[i+1:i+block_size+1] for i in random_indices])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print(x.shape, y.shape)

torch.Size([64, 256]) torch.Size([64, 256])


In [50]:
import torch.nn as nn
import torch.nn.functional as F

class FeedForward(nn.Module):
    """ a simple linear layer followed by non-linearity"""
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4* n_embed), # 4 multiplied based on the transformers paper scaled down version
            nn.ReLU(),
            nn.Linear(4*n_embed, n_embed),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Head(nn.Module):
    """one head of self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)

        #compute attention scores
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        #aggregation of values
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallel"""
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embed, n_embed) #allows the model to focus on specific aspects of the input data by projecting it into a space where relationships between different parts of the sequence can be easily assessed, making the attention mechanism more efficient and effective
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) #concatenating over the channel dimension
        out = self.proj(out)
        out = self.dropout(out)
        return out


class Block(nn.Module):
    """Transformer block: communication followed by computations"""

    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa(self.ln1(x)) #x + is the residual connection
        x = x + self.ffwd(self.ln2(x))
        return x


class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        # self.selfattention_heads = MultiHeadAttention(num_heads, n_embed//num_heads) #i.e. 4 heads of 8-dimensional self-attention
        # self.feedforward = FeedForward(n_embed)
        # self.blocks = nn.Sequential(
        #     Block(n_embed, n_head = 4),
        #     Block(n_embed, n_head = 4),
        #     Block(n_embed, n_head = 4),
        #     nn.LayerNorm(n_embed)
        # )
        self.blocks = nn.Sequential(*[Block(n_embed, n_head = num_heads) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed) #final layer norm
        self.lm_head = nn.Linear(n_embed, vocab_size)

        # better init, covered in the follow-up original GPT video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        #idx and targets are both (B, T) tensor of integers
        B, T = idx.shape
        token_embeddings = self.token_embedding_table(idx) #B, T, n_embed(C)
        position_embeddings = self.position_embedding_table(torch.arange(T, device = device)) #T, n_embed
        x = token_embeddings + position_embeddings #B, T, n_embed
        x = self.blocks(x)
        x = self.ln_f(x)
        # x = self.selfattention_heads(x) #apply one head of self-attention
        # x = self.feedforward(x)
        logits = self.lm_head(x) #B, T, vocab_size

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        idx = idx.to(device)
        for _ in range(max_new_tokens):
            #crop idx to last block_size tokens
            idx_cond = idx[:, -block_size:]
            #get predictions
            logits, loss = self(idx_cond)
            #focus only on last time step
            logits = logits[:, -1, :]
            #apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)
            #sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=-1)
        return idx



@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# logits, loss = model(x, y)

# print(logits.shape)
# print(loss)

# idx = torch.zeros((1, 1), dtype=torch.long) #starting with a newline token
# print(decode(model.generate(idx, max_new_tokens=100)[0].tolist()))




In [51]:
model = GPTLanguageModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

for steps in tqdm(range(max_iters)):
    model.train()
    xb, yb = get_batch('train')
    xb, yb = xb.to(device), yb.to(device)
    model.to(device)
    # print(device)
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if steps % eval_iters == 0 or steps == max_iters - 1:
        losses = estimate_loss()
        print(f"step {steps}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")


  0%|          | 2/2000 [01:02<14:21:14, 25.86s/it]

step 0: train loss 3.6488, val loss 3.6718


 10%|█         | 202/2000 [03:40<7:02:42, 14.11s/it] 

step 200: train loss 2.4000, val loss 2.4304


 20%|██        | 402/2000 [06:17<6:14:24, 14.06s/it]

...

step 1600: train loss 1.2503, val loss 1.5153


 90%|█████████ | 1802/2000 [24:31<46:15, 14.02s/it]  

step 1800: train loss 1.2240, val loss 1.5104


100%|██████████| 2000/2000 [27:06<00:00,  1.23it/s]

step 1999: train loss 1.1882, val loss 1.4933


In [52]:
idx = torch.zeros((1, 1), dtype=torch.long).to(device) #starting with a newline token
print(decode(model.generate(idx, max_new_tokens=1000)[0].tolist()))



OXFORDANLEY:
Ay, shall I he3 KING HENRY VI:
How now, Northumble should she dear.

DUKE OF YORK:
Senless they a conquer provered with a war,
To with ba prayers fr a joy abhoad himself
And posterity and Glod, instering kingdomy
That did ewith feast word you contented:
All mess a turn this leans almost.
The hour true is in great sworn: you have littled a word,
Win play still as i' the fair abll,
And too harm floces with a disperited of beje
I had womaning all day encounted:
Therefore the shall wealling the king his holy case,
Stand from my mornsal thanks malices,
Though the seople doth good, dost general.

First Servant:
O hath remedy, and loathed was the lack,
Or, these weed from the treasons satoch
Are ne'er feel. Although, seeing hates been prival,
One a dancoured nobour?

MENENIUS:
My brave is answer my heads!
And would yet here's a prated I say; I'll keep
The edge pity for Cominius, and rememberal.

MENIUS:
When it
Was datches; for thick? that's treamily, another neither?

MEssenge

In [53]:
#save the model
torch.save(model.state_dict(), 'model.pth')

In [54]:
import numpy as np

-np.log(1/65)
#this is the expected loss for a random guess,
# but since we are getting 4.27 it means our model has some extra entropy and the loss is higher than expected


4.174387269895637

# Attention

In [55]:
 #toy example

torch.manual_seed(1337)
B,T,C = 4,8,32

x = torch.randn(B,T,C)

# single head self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) #(B,T,16)
q = query(x) #(B, T, 16)
v = value(x) #(B, T, 16)

weights = q @ k.transpose(-2, -1) #(B,T,16) @ (B,16,T) -> (B,T,T)

tril = torch.tril(torch.ones(T,T))
# weights = torch.zeros(T,T)
weights = weights.masked_fill(tril==0, float('-inf'))
weights = F.softmax(weights, dim=-1)
out = weights @ v

out.shape


torch.Size([4, 8, 16])

In [56]:
weights

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089